# BEAST2
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/EnzoAndree/ColabBEAST/blob/main/BEAST2.ipynb)

Notebook Version = 1.0.0

To report issues, comments or improvements you can do it in the [EnzoAndree/ColabBEAST](https://github.com/EnzoAndree/ColabBEAST) repository.

Made with ❤️ by [@eguerreroaraya](https://twitter.com/eguerreroaraya)

In [ ]:
#@title Check Nvidia GPU
import ipywidgets as widgets
import pandas as pd
import psutil
from time import sleep, time
from bokeh.resources import INLINE
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook, curdoc
from bokeh.themes import built_in_themes
from bokeh.models import BasicTickFormatter, Legend, NumeralTickFormatter
from pathlib import Path
import shlex, subprocess

def get_compute(name):
  if 'A100' in name:
    return '80'
  elif 'V100' in name:
    return '70'
  elif 'P4' in name:
    return '61'
  elif 'T4' in name:
    return '75'
  elif 'P100' in name:
    return '60'
  elif 'K80' in name:
    return '37'
def get_packagemanager_list(output):
  lines = outpack[4:]
  listofpack = []
  for package in lines:
    listofpack.append(package.split()[0])
  return listofpack

!nvidia-smi
Gcardinfo = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv,noheader,nounits
Gcardname, Gcardram = Gcardinfo[0].split(', ')

In [ ]:
%%capture
#@title Install BEAST2, Beagle, Beastiary
!pip install pyngrok beastiary
checkinstall = Path('./BEAST2Beagle_READY')
if not checkinstall.is_file():
    !wget https://github.com/CompEvol/beast2/releases/download/v2.6.4/BEAST.v2.6.4.Linux.tgz
    !tar -zxvf BEAST.v2.6.4.Linux.tgz
    !rm -fr BEAST.v2.6.4.Linux.tgz
    %cd /content
    !apt-get install build-essential autoconf automake libtool git pkg-config
    !git clone --depth=1 https://github.com/beagle-dev/beagle-lib.git
    %cd beagle-lib
    # http://arnon.dk/matching-sm-architectures-arch-and-gencode-for-various-nvidia-cards/
    # Nvidia A100 compute_80
    # Nvidia V100 compute_70
    # Nvidia P4 compute_61
    # Nvidia T4 compute_75
    # Nvidia P100 compute_60
    # Nvidia K80 compute_37
    !sed -i 's/-arch compute_30/-gencode=arch=compute_{get_compute(Gcardname)},code=sm_{get_compute(Gcardname)}/' configure.ac
    !./autogen.sh
    !./configure --prefix=$HOME
    !make install
    !make check
    %env LD_LIBRARY_PATH=/root/lib
    %cd /content
    !touch BEAST2Beagle_READY

In [ ]:
#@title Install BEAST2 modules
# !./beast/bin/packagemanager -add CoupledMCMC
# !./beast/bin/packagemanager -add bacter
# !./beast/bin/packagemanager -add NS
outpack = !./beast/bin/packagemanager -list
modlist = get_packagemanager_list(outpack)
typocheck = {x.lower(): x for x in modlist}
modules = '' #@param {type:"string"}
#@markdown - `modules` Specify the extra modules to be installed separated by commas. Leave it blank if you do not need extra modules.
#@markdown  - Use `!./beast/bin/packagemanager -list` to get a list of modules availables. 

to_install = []
if modules != '':
  errorfound = False
  modules = modules.split(',')
  modules = [m.strip() for m in modules]
  for m in modules:
    if m.lower() in typocheck.keys():
      to_install.append(typocheck[m.lower()])
    else:
      errorfound = True
      print(f'{m} is not found in the modulule list {modlist}')
      break
  if not errorfound:
    print(f'This modules will be installed: {to_install}')
    for m in to_install:
      !./beast/bin/packagemanager -add {m}
  


In [ ]:
#@title Check Beagle resources
!./beast/bin/beast -beagle_info 

In [ ]:
#@title Run BEAST2
curdoc().theme = 'dark_minimal'

Path_to_XML = 'beast/examples/testGTR.xml' #@param {type:"string"}
upload_custom_XML = False #@param {type:"boolean"}
job_name = '4MCtest' #@param {type:"string"}
parallel_jobs =  4 #@param {type:"integer"}
resume = False #@param {type:"boolean"}
overwrite = True #@param {type:"boolean"}
beagle = True #@param {type:"boolean"}
beagle_GPU = True #@param {type:"boolean"}
beagle_double = True #@param {type:"boolean"}

if upload_custom_XML:
  print(f'upload custom XML')
  XML = files.upload()
  Path_to_XML = str(list(XML.keys())[0])
  # print(XML[list(XML.keys())[0]], file=open(list(XML.keys())[0], 'w'))



# Make directories
output_dirs = []
pids = []
procs = []
for run in range(1, parallel_jobs+1):
  name = f'{job_name}_MC{run}_'
  output_dirs.append(name)
  !mkdir -p {name}
  # Make CLI
  cli = './beast/bin/beast'
  if resume:
    cli += ' -resume'
  if overwrite:
    cli += ' -overwrite'
  if beagle:
    cli += ' -beagle'
  if beagle_GPU:
    cli += ' -beagle_GPU'
  if beagle_double:
    cli += ' -beagle_double'
  cli += f' -prefix {job_name}_MC{run}_ {Path_to_XML}'
  print(cli)
  stdout = open(f'{job_name}_MC{run}.out.logger', 'wb')
  stderr = open(f'{job_name}_MC{run}.err.logger', 'wb')
  proc = subprocess.Popen(shlex.split(cli), stdout=stdout, stderr=stderr)
  pids.append(proc.pid)
  procs.append(psutil.Process(proc.pid))


print(f'Running {parallel_jobs} BEAST: {output_dirs}; PID: {pids}')
#print('Please wait 3 seconds')
#sleep(3)

In [ ]:
#@title Show the chains logs with [beastiary](https://github.com/Wytamma/beastiary)

from IPython.display import Javascript
from pyngrok import ngrok

def show_beastiary(src, height=600):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = '%s'
    fm.width = '95%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (src, height) ))

# kill beastiary is it's already running
import os
import signal
from time import sleep
from glob import glob

path_to_log_files = '/content/*.log' #@param {type:"string"}

while True:
  # wait for log files
  sleep(1)
  f = [n for n in glob(path_to_log_files) if os.path.isfile(n)]
  if f:
    break

# stop beastiary if it's already running 
try:
  beastiary_proc = !ps | grep 'beastiary$' 
  beastiary_pid = int(beastiary_proc[0].split()[0])
  os.kill(beastiary_pid, 9)
except:
  pass
  
cli = f'beastiary --no-security --port 34345 {path_to_log_files} &'
get_ipython().system_raw(cli) 

# wait for server to start
print("Waitng for beastiary to start...")
sleep(5)
tunnels = ngrok.get_tunnels()
if not tunnels:
  tunnel = ngrok.connect(34345)
else:
  tunnel = tunnels[0]

print(tunnel.public_url)
show_beastiary(tunnel.public_url)

In [ ]:
#@title Download results
from google.colab import files
wkd = Path('./')
outputfiles = wkd.glob(f'{job_name}*')
to_download = ' '.join([x.name for x in list(outputfiles) if not x.name.endswith('.zip')])

!zip -FSr $job_name'.zip' $to_download
files.download(f'{job_name}.zip')